In [1]:
import json
import requests
import re
from IPython import display
from datetime import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import openpyxl

In [2]:
FIREBASE_PRIVATE_KEY = r'C:\Users\johnp\Documents\ESportsAnalytics\esportsanalytics-3b04cf0c1d47.json'
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'


In [3]:
# Connect to Firebase and return a database instance
def connecttofirebase(PATH):
    
    # Check if the app is already initialised, if not then connect
    if not firebase_admin._apps:
        cred = credentials.Certificate(FIREBASE_PRIVATE_KEY)
        firebase_admin.initialize_app(cred)
    
    # Get an instance of the project database
    db = firestore.client()
    
    return db

In [4]:
db = connecttofirebase(FIREBASE_PRIVATE_KEY)

In [5]:
db.project

'esportsanalytics'

# Data Extraction

In [6]:
def get_all_teams_from_firebase():
    
     # get the data from Firebase
    doc_ref = db.collection(u'dota').document(u'all_teams')
    doc = doc_ref.get()
    
    # get the upload time from Firebase
    datetime_upload_str = doc.to_dict()['date_time_upload']
    datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")

    # subtracting two datetime objects result in a datetime.timedelta object
    data_age_timedelta = datetime.now() - datetime_upload
    data_age_in_days = data_age_timedelta.total_seconds() / 86400
    
    print('Data is:', data_age_in_days, 'old')
    
    # Get the data from Firebase
    team_data_json = doc.to_dict()['all_teams_info']
    
    return team_data_json

In [7]:
def get_all_teams_from_opendota():
    
    response = requests.get(BASE_URL + TEAMS)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [8]:
def update_all_teams_in_firebase():
    
    response = requests.get(BASE_URL + TEAMS)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:
        # upload the team_data_json in Firebase if there are no errors
        data = {
            u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
            u'all_teams_info' : team_data_json
        }

        db.collection(u'dota').document(u'all_teams').set(data)
        print("All Teams data updated in Firebase")

In [9]:
# Save the JSON as a file in the working directory
def save_all_teams_data_to_file(all_teams_data_json):
    
    with open('all_teams_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_teams_data_json, f, ensure_ascii=False, indent=4)

In [10]:
def get_all_heroes_from_firebase():
    
    # get the data from Firebase
    doc_ref = db.collection(u'dota').document(u'all_heroes')
    doc = doc_ref.get()
    
    # get the upload time from Firebase
    datetime_upload_str = doc.to_dict()['date_time_upload']
    datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")

    # subtracting two datetime objects result in a datetime.timedelta object
    data_age_timedelta = datetime.now() - datetime_upload
    data_age_in_days = data_age_timedelta.total_seconds() / 86400
    
    print('Data is:', data_age_in_days, 'old')
    
    # Get the data from Firebase
    heroes_data_json = doc.to_dict()['all_heroes_info']
    
    return heroes_data_json

In [11]:
def get_all_heroes_from_opendota():
    response = requests.get(BASE_URL + HEROES)

    try:
        heroes_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return heroes_data_json
    
    return 0

In [12]:
def update_all_heroes_in_firebase():
    
    response = requests.get(BASE_URL + HEROES)

    try:
        heroes_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:
        # upload the team_data_json in Firebase if there are no errors
        data = {
            u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
            u'all_teams_info' : heroes_data_json
        }

        db.collection(u'dota').document(u'all_heroes').set(data)
        print("All Heroes data updated in Firebase")

In [13]:
# Save the JSON as a file in the working directory
def save_all_heroes_data_to_file(all_heroes_data_json):
    
    with open('all_heroes_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_heroes_data_json, f, ensure_ascii=False, indent=4)

In [14]:
def get_all_pro_players_in_opendota():
    
    response = requests.get(BASE_URL + PROPLAYERS)

    try:
        all_pro_players_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return all_pro_players_data_json
    
    return 0

In [15]:
def get_all_pro_players_in_firebase():
    all_pro_players_data_json = []
    
    firebase_doc_list = [
        u'all_pro_players_1',
        u'all_pro_players_2',
        u'all_pro_players_3'
    ]
    
    for index, item in enumerate(firebase_doc_list):
        
        doc_ref = db.collection(u'dota').document(item)
        doc = doc_ref.get()
        
        # Get the data from Firebase
        all_pro_players_data_json += doc.to_dict()['all_pro_players_info_'+str(index+1)]
    
    return all_pro_players_data_json

In [16]:
def update_all_pro_players_in_firebase():

    # Do these if data doesn't exist yet or is not up to date
    response = requests.get(BASE_URL + PROPLAYERS)

    try:
        pro_players_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    else:

        pro_players_split = [
            {u'all_pro_players_info_1' : pro_players_data_json[0:1000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
            {u'all_pro_players_info_2' : pro_players_data_json[1000:2000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
            {u'all_pro_players_info_3' : pro_players_data_json[2000:], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},

        ]

        for index, data in enumerate(pro_players_split):
            pro_players_ref = db.collection(u'dota').document(u'all_pro_players_'+str(index+1)).set(data)

        print("All Pro Players data updated in Firebase")
    

In [17]:
# Save the JSON as a file in the working directory
def save_all_pro_players_data_to_file(all_pro_players_data_json):
    
    with open('all_pro_players_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_pro_players_data_json, f, ensure_ascii=False, indent=4)

In [18]:
def get_opendota_api_info():
    
    response = requests.get(BASE_URL)

    try:
        opendota_api_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return opendota_api_data_json
    
    return 0

In [19]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id))

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [20]:
def get_all_heroes_stats():
    response = requests.get(BASE_URL + HEROES_STATS)

    try:
        all_heroes_stats_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return all_heroes_stats_data_json
    
    return 0

# Data Search

In [21]:
def search_regex(search_string, name_from_list):
    try:
        x = re.search(search_string, name_from_list, re.IGNORECASE).group()
    except Exception:
        return False
    return x

In [22]:
def search_item(item_to_search, item_key, data_source):
    item_filter =  filter(lambda y: search_regex(item_to_search, y[item_key]), data_source)
    return [item_info for item_info in item_filter]

In [23]:
all_teams = get_all_teams_from_opendota()

In [24]:
x = search_item('fnatic', 'name', all_teams)
x[0]

{'team_id': 350190,
 'rating': 1403.13,
 'wins': 1153,
 'losses': 845,
 'last_match_time': 1661607736,
 'name': 'Fnatic',
 'tag': 'Fnatic',
 'logo_url': 'https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/350190.png'}

In [25]:
def get_team_matches(team_id):
    
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + MATCHES)
    print(response)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [26]:
team_fnatic_matches = get_team_matches(350190)

<Response [200]>


In [27]:
len(team_fnatic_matches)

2081

In [28]:
team_og_matches = get_team_matches(2586976)

<Response [200]>


In [79]:
team_og_matches[0]

{'match_id': 6728572058,
 'radiant_win': True,
 'radiant': True,
 'duration': 2455,
 'start_time': 1661683040,
 'leagueid': 14388,
 'league_name': 'ESL One Malaysia 2022 powered by Intel',
 'cluster': 155,
 'opposing_team_id': 6209166,
 'opposing_team_name': 'Team Aster',
 'opposing_team_logo': 'https://steamusercontent-a.akamaihd.net/ugc/1629730253163462144/FEF15B2B4528E81103DD693E37FD7B6C9BE6B471/'}

with open('team_og_matches.json', 'w', encoding='utf-8') as f:
    json.dump(team_og_matches, f, ensure_ascii=False, indent=4)

In [30]:
all_heroes = get_all_heroes_from_opendota()

In [31]:
all_pro_players = get_all_pro_players_in_opendota()

In [32]:
def match_outcome(radiant_win, radiant):
    if ((radiant_win == False and radiant == False) or (radiant_win == True and radiant == True)):
        return True
    
    elif ((radiant_win == True and radiant == False) or (radiant_win == False and radiant == True)):
        return False

In [33]:
# Convert the team matches json to a dataframe
def team_matches_df(team_matches_json):
    
    all_matches_df = pd.DataFrame(data = [(datetime.fromtimestamp(item['start_time']).strftime('%Y-%m-%d %H:%M:%S'), 
                                                       item['opposing_team_name'],
                                                       item['opposing_team_id'], 
                                                       item['league_name'],
                                                       item['leagueid'],
                                                       item['cluster'], 
                                                       item['radiant_win'],
                                                       item['radiant'],
                                                       item['duration'], 
                                                       item['match_id'],
                                                       match_outcome(item['radiant_win'], item['radiant'])) for item in team_matches_json], 
                      columns = ['start_date', 'opposing_team_name', 'opposing_team_id', 'league_name', 'leagueid', 'cluster', 'radiant_win', 'radiant', 'duration', 'match_id', 'match_outcome'])
    
    return all_matches_df


In [34]:
team_og_matches_df = team_matches_df(team_og_matches)

In [120]:
og_v_fnatic_match_info = get_match_info(6728572058)

In [121]:
len(og_v_fnatic_match_info)

46

In [122]:
for key in og_v_fnatic_match_info:
    print(key)

match_id
barracks_status_dire
barracks_status_radiant
chat
cluster
cosmetics
dire_score
dire_team_id
draft_timings
duration
engine
first_blood_time
game_mode
human_players
leagueid
lobby_type
match_seq_num
negative_votes
objectives
picks_bans
positive_votes
radiant_gold_adv
radiant_score
radiant_team_id
radiant_win
radiant_xp_adv
skill
start_time
teamfights
tower_status_dire
tower_status_radiant
version
replay_salt
series_id
series_type
league
radiant_team
dire_team
players
patch
region
all_word_counts
my_word_counts
throw
loss
replay_url


In [123]:
players = og_v_fnatic_match_info['players']

In [124]:
len(players)

10

In [125]:
[p_keys for p_keys in players[0]]

['match_id',
 'player_slot',
 'ability_targets',
 'ability_upgrades_arr',
 'ability_uses',
 'account_id',
 'actions',
 'additional_units',
 'assists',
 'backpack_0',
 'backpack_1',
 'backpack_2',
 'backpack_3',
 'buyback_log',
 'camps_stacked',
 'connection_log',
 'creeps_stacked',
 'damage',
 'damage_inflictor',
 'damage_inflictor_received',
 'damage_taken',
 'damage_targets',
 'deaths',
 'denies',
 'dn_t',
 'firstblood_claimed',
 'gold',
 'gold_per_min',
 'gold_reasons',
 'gold_spent',
 'gold_t',
 'hero_damage',
 'hero_healing',
 'hero_hits',
 'hero_id',
 'item_0',
 'item_1',
 'item_2',
 'item_3',
 'item_4',
 'item_5',
 'item_neutral',
 'item_uses',
 'kill_streaks',
 'killed',
 'killed_by',
 'kills',
 'kills_log',
 'lane_pos',
 'last_hits',
 'leaver_status',
 'level',
 'lh_t',
 'life_state',
 'max_hero_hit',
 'multi_kills',
 'net_worth',
 'obs',
 'obs_left_log',
 'obs_log',
 'obs_placed',
 'party_id',
 'party_size',
 'performance_others',
 'permanent_buffs',
 'pings',
 'pred_vict',
 

In [126]:
team_og_matches_df.head()

,start_date,opposing_team_name,opposing_team_id,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
0,2022-08-28 20:37:20,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,155,True,True,2455,6728572058,True
1,2022-08-28 19:19:19,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,155,False,False,2996,6728463079,True
2,2022-08-28 18:03:26,Team Aster,6209166,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,2776,6728333090,True
3,2022-08-28 15:18:42,Team Secret,1838315,ESL One Malaysia 2022 powered by Intel,14388,151,False,False,1628,6728136960,True
4,2022-08-28 14:04:20,Team Secret,1838315,ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2547,6728055442,True


In [127]:
for i in range(10):
    
    print('name:', players[i]['name'], 'lane role:', players[i]['lane_role'], 'accountid:',  players[i]['account_id'], 'lane efficiency:', players[i]['lane_efficiency_pct'], 'kda:', players[i]['kda'],'rank_tier:', players[i]['rank_tier'], 'is radiant:', players[i]['isRadiant'])

name: AMMAR_THE_F lane role: 3 accountid: 183719386 lane efficiency: 78 kda: 4 rank_tier: 80 is radiant: True
name: bzm lane role: 2 accountid: 93618577 lane efficiency: 61 kda: 6 rank_tier: 80 is radiant: True
name: Yuragi lane role: 1 accountid: 167976729 lane efficiency: 69 kda: 8 rank_tier: 80 is radiant: True
name: Chu lane role: 3 accountid: 117483894 lane efficiency: 27 kda: 7 rank_tier: 80 is radiant: True
name: Taiga lane role: 1 accountid: 401792574 lane efficiency: 33 kda: 4 rank_tier: 80 is radiant: True
name: Monet lane role: 1 accountid: 148215639 lane efficiency: 66 kda: 2 rank_tier: 80 is radiant: False
name: Xxs lane role: 3 accountid: 129958758 lane efficiency: 80 kda: 1 rank_tier: 80 is radiant: False
name: BoBoKa lane role: 3 accountid: 207829314 lane efficiency: 35 kda: 1 rank_tier: 80 is radiant: False
name: Ori lane role: 2 accountid: 107803494 lane efficiency: 80 kda: 0 rank_tier: 80 is radiant: False
name: 皮球 lane role: 1 accountid: 183378746 lane efficiency: 2

In [88]:
taiga = get_player_info('401792574')
taiga

<Response [200]>


{'solo_competitive_rank': 6261,
 'competitive_rank': None,
 'leaderboard_rank': 48,
 'mmr_estimate': {'estimate': 6386},
 'profile': {'account_id': 401792574,
  'personaname': '?',
  'name': 'Taiga',
  'plus': True,
  'cheese': 0,
  'steamid': '76561198362058302',
  'avatar': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1.jpg',
  'avatarmedium': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_medium.jpg',
  'avatarfull': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/Taigadota/',
  'last_login': '2017-10-31T05:53:29.049Z',
  'loccountrycode': 'NO',
  'status': None,
  'is_contributor': False,
  'is_subscriber': False},
 'rank_tier': 80}

In [128]:
def get_player_account_id_from_match_id(match_id, isRadiant):
    opposing_team = []
    prediction_team = []
    
    match_info_json = get_match_info(match_id)
    all_players = match_info_json['players']
    
    for i in range(10):
        player_info = get_player_info(all_players[i]['account_id'])
        
        temp = {'name' : all_players[i]['name'],
                'account_id' : all_players[i]['account_id'],
                'mmr_estimate' : player_info['mmr_estimate']['estimate'],
                'rank_tier' : player_info['rank_tier'],
                'leaderboard_rank' : player_info['leaderboard_rank'],
                'solo_competitive_rank' : player_info['solo_competitive_rank']}
                                      
        # if the 'isRadiant' is equal to isRadiant then the player is on the prediction team
        if (all_players[i]['isRadiant'] == isRadiant):
                                          
            prediction_team.append(temp)
        else:                          
            opposing_team.append(temp)
            
    return {'opposing_team' : opposing_team, 'prediction_team' : prediction_team}

In [46]:
def get_team_players(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + PLAYERS)
    print(response)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

In [47]:
def get_player_info(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id))
    print(response)

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [48]:
def get_player_peers(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id) + '/peers')
    print(response)

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [49]:
team_og_players = get_team_players(2586976)

<Response [200]>


In [50]:
team_og_players[0]

{'account_id': 19672354,
 'name': 'N0tail',
 'games_played': 1150,
 'wins': 709,
 'is_current_team_member': False}

In [51]:
player = search_item('taiga', 'name', all_pro_players)

In [52]:
[item for item in all_heroes if item['id'] == 74]

[{'id': 74,
  'name': 'npc_dota_hero_invoker',
  'localized_name': 'Invoker',
  'primary_attr': 'int',
  'attack_type': 'Ranged',
  'roles': ['Carry', 'Nuker', 'Disabler', 'Escape', 'Pusher'],
  'legs': 2}]

In [53]:
player

[{'account_id': 401792574,
  'steamid': '76561198362058302',
  'avatar': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1.jpg',
  'avatarmedium': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_medium.jpg',
  'avatarfull': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/Taigadota/',
  'personaname': '?',
  'last_login': '2017-10-31T05:53:29.049Z',
  'full_history_time': '2022-08-27T17:40:32.077Z',
  'cheese': 0,
  'fh_unavailable': True,
  'loccountrycode': 'NO',
  'last_match_time': '2022-08-28T10:37:20.000Z',
  'plus': True,
  'name': 'Taiga',
  'country_code': 'no',
  'fantasy_role': 2,
  'team_id': 2586976,
  'team_name': 'OG',
  'team_tag': 'OG',
  'is_locked': True,
  'is_pro': True,
  'locked_until': None}]

In [54]:
taiga_account_id = 401792574

# What makes a team win matches?

-players? Team composition

-draft pick? Hero composition

-items? Item-Hero compatibility

-time? Match duration

# What makes a specific team win?

Since you can't include team names in the dataset, teams should be represented numerically with information that excludes historical match performance data (because new data pre-hero-draft won't include match data of course)

The only way I can think of is to use that specific match's player's historical performance data leading up to that match as a representation of the teams e.g. 

Team OG Matches from 2015:

	    start_date	        opposing_team_name	    opposing_team_id	league_name	leagueid	        cluster	radiant_win	radiant	duration	match_id	match_outcome
        
1328	2015-11-04 04:50:57	MONKEY FREEDOM FIGHTERS	758797	            ASUS ROG DreamLeague Season 4	3865	133	True	True	1423	1912135868	True

1329	2015-11-03 08:09:58	Mamas-Boys	            2276247	            ASUS ROG DreamLeague Season 4	3865	138	True	True	1092	1910208777	True

For match id 1912135868, include all 5 player performance data from the lead up to that 2015 match and don't include future data beyond 2015 even if we have it!

# Figure out which data to use to quantify player performance

e.g. lane efficiency, MMR, years experience, team cohesion

# Figure out which data to use to quantify team performance

e.g. team fight success rate, team brand goodwill? Team Rank

1. Get Team A vs B Match Info --> Get Players --> Get Player Account IDs --> Get Historical Team Win Rate together

In [55]:
opendota_api_info = get_opendota_api_info()

In [87]:
taiga = get_player_info('401792574')
taiga

<Response [200]>


{'profile': {'account_id': 401792574,
  'personaname': '?',
  'name': 'Taiga',
  'plus': True,
  'cheese': 0,
  'steamid': '76561198362058302',
  'avatar': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1.jpg',
  'avatarmedium': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_medium.jpg',
  'avatarfull': 'https://avatars.akamai.steamstatic.com/fede1355e88a5ffae6e9bfc20de4ec33a18ef5d1_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/Taigadota/',
  'last_login': '2017-10-31T05:53:29.049Z',
  'loccountrycode': 'NO',
  'status': None,
  'is_contributor': False,
  'is_subscriber': False},
 'competitive_rank': None,
 'rank_tier': 80,
 'leaderboard_rank': 48,
 'mmr_estimate': {'estimate': 6386},
 'solo_competitive_rank': 6261}

In [75]:
ammar = get_player_info(183719386)
print(ammar['mmr_estimate']['estimate'])
print(ammar['leaderboard_rank'])

<Response [200]>
5675
14


In [58]:
taiga_peers = get_player_peers(401792574)
[item for item in taiga_peers if item['account_id'] == 183719386]

<Response [200]>


[{'account_id': 183719386,
  'last_played': 1661683040,
  'win': 113,
  'games': 186,
  'with_win': 106,
  'with_games': 173,
  'against_win': 7,
  'against_games': 13,
  'with_gpm_sum': 53927,
  'with_xpm_sum': 74213,
  'personaname': 'ATM',
  'name': 'AMMAR_THE_F',
  'is_contributor': False,
  'is_subscriber': False,
  'last_login': '2021-05-05T15:49:21.412Z',
  'avatar': 'https://avatars.akamai.steamstatic.com/6dde62ad2f5e6d32c9d0b7432313ce82c189bf30.jpg',
  'avatarfull': 'https://avatars.akamai.steamstatic.com/6dde62ad2f5e6d32c9d0b7432313ce82c189bf30_full.jpg'}]

In [ ]:
opendota_api_info

In [78]:
team_og_matches_df[team_og_matches_df.opposing_team_id == 350190]

,start_date,opposing_team_name,opposing_team_id,league_name,leagueid,cluster,radiant_win,radiant,duration,match_id,match_outcome
5,2022-08-27 23:42:16,Fnatic,350190,ESL One Malaysia 2022 powered by Intel,14388,152,False,False,1931,6727040019,True
6,2022-08-27 22:25:20,Fnatic,350190,ESL One Malaysia 2022 powered by Intel,14388,156,False,False,2719,6726897362,True
7,2022-08-27 21:16:45,Fnatic,350190,ESL One Malaysia 2022 powered by Intel,14388,155,False,True,2252,6726755759,False
20,2022-08-23 22:50:01,Fnatic,350190,ESL One Malaysia 2022 powered by Intel,14388,156,True,True,1635,6720651873,True
21,2022-08-23 21:52:01,Fnatic,350190,ESL One Malaysia 2022 powered by Intel,14388,152,False,False,1537,6720571150,True
50,2022-08-05 02:13:29,Fnatic,350190,PGL Arlington Major 2022,14417,121,False,False,1941,6692354598,True
51,2022-08-05 01:08:00,Fnatic,350190,PGL Arlington Major 2022,14417,122,True,False,2015,6692259411,False
90,2022-05-21 02:35:09,Fnatic,350190,ESL One Stockholm Major 2022 powered by Intel,14173,184,False,False,3846,6578909443,True
91,2022-05-21 01:25:10,Fnatic,350190,ESL One Stockholm Major 2022 powered by Intel,14173,182,False,True,2181,6578799684,False
92,2022-05-21 00:09:48,Fnatic,350190,ESL One Stockholm Major 2022 powered by Intel,14173,187,False,False,2698,6578674296,True


In [76]:
team_og_matches_df.shape

(1335, 11)

In [129]:
teams = get_player_account_id_from_match_id(6727040019, False)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [132]:
teams['prediction_team']

[{'name': 'AMMAR_THE_F',
  'account_id': 183719386,
  'mmr_estimate': 5675,
  'rank_tier': 80,
  'leaderboard_rank': 14,
  'solo_competitive_rank': 3333},
 {'name': 'Chu',
  'account_id': 117483894,
  'mmr_estimate': 6368,
  'rank_tier': 80,
  'leaderboard_rank': 314,
  'solo_competitive_rank': 7693},
 {'name': 'bzm',
  'account_id': 93618577,
  'mmr_estimate': 6268,
  'rank_tier': 80,
  'leaderboard_rank': 10,
  'solo_competitive_rank': 5338},
 {'name': 'Yuragi',
  'account_id': 167976729,
  'mmr_estimate': 6233,
  'rank_tier': 80,
  'leaderboard_rank': 6,
  'solo_competitive_rank': 5351},
 {'name': 'Taiga',
  'account_id': 401792574,
  'mmr_estimate': 6386,
  'rank_tier': 80,
  'leaderboard_rank': 48,
  'solo_competitive_rank': 6261}]